# Uzyskaj dostęp do danych nt. COVID-19 z poziomu Pythona.

In [1]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="pro-landing-415716-4990cdcfad2f.json"
client = bigquery.Client() 

In [2]:
query = ("SELECT * FROM bigquery-public-data.covid19_open_data.covid19_open_data LIMIT 10 ")

query_job = client.query(query)    
query_result = query_job.result()  
dfC = query_result.to_dataframe()

# 3 Misja dodatkowa - Poznanie danych

Zapoznaj się z informacjami o danych na temat COVID-19, m.in.:

## 3.1. Sprawdź, ile jest zapisanych wierszy z danymi. 

In [3]:
query = ("SELECT COUNT(*) AS number_of_rows FROM bigquery-public-data.covid19_open_data.covid19_open_data")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,number_of_rows
0,22756333


## 3.2. Sprawdź, ile krajów jest uwzględnionych w danych.

In [4]:
query = ("SELECT COUNT(DISTINCT location_key) AS number_of_countries FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE aggregation_level = 0")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,number_of_countries
0,246


## 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.

In [5]:
query = ("SELECT * FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE iso_3166_1_alpha_2 = 'BR' AND date = '2020-01-01'")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,BR_BA_291050,2020-01-01,ChIJ_95QKDTTFgcRXsCBfygbREc,Q1806968,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-38.083889 -11.941944)
1,BR_BA_291995,2020-01-01,ChIJ7URnZsC_RQcRWtOYQs2ZcFE,Q1794883,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-41.491944 -14.662778)
2,BR_BA_292250,2020-01-01,ChIJL6K8GMshPgcRl8DPLYlxy88,Q1808393,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-39.013889 -13.035)
3,BR_BA_292580,2020-01-01,ChIJfzDBbtTYEgcR0mNlt9jY5W4,Q1808402,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-39.623889 -10.977778)
4,BR_BA_292780,2020-01-01,ChIJR_OKYFFtOAcRuaBjEgReYWE,Q1761901,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-39.811944 -14.960833)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,BR_SP_354370,2020-01-01,ChIJOfZKgOVVuJQR3ESV63iqlpQ,Q744399,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-48.070833 -21.586944)
5612,BR_SP_354520,2020-01-01,ChIJ7frs2GVSz5QRp5SW3cXtdOc,Q1994826,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-47.286944 -23.200833)
5613,BR_SP_355270,2020-01-01,ChIJH3kVg4rWuJQREp3JSitgXdA,Q22064419,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-48.687778 -21.716944)
5614,BR_TO_171245,2020-01-01,ChIJW8uFxj3u2pIRKJh7zZ7tkFY,Q1801642,None,BR,Brazil,BR,BRA,2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-47.855833 -6.181944)


Dla każdego kraju w każdym dniu zapisywana jest jedna wartość przez okres 991 dni jednak niektóre kraje mają uwzględnione również dane 
z mniejszych jednostek administracyjnych w wyniku czego dla niektórych krajów tworzy się więcej wartości.

Mniejsze jednostki administracyjne oddziolone są podłogą i dalszymi znakami. Biorąc jedynie rekody gdzie aggregation_level był równy 0 ograniczamy zapytanie jedynie do zgrupowanych danych na temat danych krajów. W wyniku tego zapytania liczba country code'ów ograniczona jest do 246 czyli do libczy krajów w zbiorze danych, natomiast liczba rekorów zmniejsza się do 243786.

In [6]:
query = ("SELECT iso_3166_1_alpha_2, COUNT(*) AS number_of_subregions FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE date = '2020-01-01' GROUP BY iso_3166_1_alpha_2")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,iso_3166_1_alpha_2,number_of_subregions
0,TO,1
1,BG,29
2,TW,23
3,MO,1
4,UA,27
...,...,...
241,LU,1
242,BW,1
243,LI,1
244,KG,1


## 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.
- Wartości całkowite zapisywane są bez żadnych separatorów.
- Wartości zmiennoprzecinkowe są zapisane z użyciem kropki.
- W zbiorze znajdują się wartości 'per 1000' zapisane są one również jako zmiennoprzecinkowe.
- W zbiorze znajduje się również jedna kolumna z typem datowym i jedna z typem geography

In [7]:
query = ("SELECT column_name, data_type FROM bigquery-public-data.covid19_open_data.INFORMATION_SCHEMA.COLUMNS")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,column_name,data_type
0,province_state,STRING
1,country_region,STRING
2,date,DATE
3,latitude,FLOAT64
4,longitude,FLOAT64
...,...,...
710,new_persons_fully_vaccinated_janssen,INT64
711,cumulative_persons_fully_vaccinated_janssen,INT64
712,new_vaccine_doses_administered_janssen,INT64
713,cumulative_vaccine_doses_administered_janssen,INT64


## 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [8]:
query = ("SELECT MIN(date) as min_date, MAX(date) as max_date FROM bigquery-public-data.covid19_open_data.covid19_open_data")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,min_date,max_date
0,2020-01-01,2022-09-17


Okres z jakiego pochodzą wartości na temat zachorowań, śmierci oraz zaszczepionych różnią się dla poszczególnych krajów, ponieważ kraje w różnych okresach zaczęły i przestały podawać informacje.

dane o zarażonych:

In [9]:
query = ("SELECT MIN(date) as min_date, MAX(date) as max_date FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE new_confirmed IS NOT NULL")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,min_date,max_date
0,2020-01-01,2022-09-16


dane o śmierciach:

In [10]:
query = ("SELECT MIN(date) as min_date, MAX(date) as max_date FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE new_deceased IS NOT NULL")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,min_date,max_date
0,2020-01-01,2022-09-16


dane o zaszczepionych:

In [11]:
query = ("SELECT MIN(date) as min_date, MAX(date) as max_date FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE new_persons_vaccinated IS NOT NULL")

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,min_date,max_date
0,2020-03-12,2022-09-16


## 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 

### 3.6.1 W jakich okresach czasu poszczególne państwa udostępniały informacje o zachorowaniach

In [12]:
query = ('''SELECT iso_3166_1_alpha_2, country_name, MIN(date) as min_date, MAX(date) as max_date 
            FROM bigquery-public-data.covid19_open_data.covid19_open_data 
            WHERE new_confirmed IS NOT NULL GROUP BY iso_3166_1_alpha_2, country_name ORDER BY min_date DESC, max_date ASC''')

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,iso_3166_1_alpha_2,country_name,min_date,max_date
0,MO,Macau,2020-02-20,2022-09-02
1,HK,Hong Kong,2020-01-23,2022-02-06
2,TW,Taiwan,2020-01-22,2022-09-15
3,KI,Kiribati,2020-01-02,2022-09-14
4,TO,Tonga,2020-01-02,2022-09-14
...,...,...,...,...
227,FR,France,2020-01-01,2022-09-15
228,BR,Brazil,2020-01-01,2022-09-15
229,ID,Indonesia,2020-01-01,2022-09-15
230,AU,Australia,2020-01-01,2022-09-16


### 3.6.2 Jaki procent ludności w poszczególnych grupach wiekowych znajduje sie w każdym z krajów

In [13]:
query = ('''SELECT DISTINCT location_key, country_name, 
            population_age_00_09/population*100 AS age_00_09_percentage,
            population_age_10_19/population*100 AS age_10_19_percentage, 
            population_age_20_29/population*100 AS age_20_29_percentage, 
            population_age_30_39/population*100 AS age_30_39_percentage, 
            population_age_40_49/population*100 AS age_40_49_percentage, 
            population_age_50_59/population*100 AS age_50_59_percentage, 
            population_age_60_69/population*100 AS age_60_69_percentage, 
            population_age_70_79/population*100 AS age_70_79_percentage, 
            population_age_80_and_older/population*100 AS age_80_plus_percentage
            FROM bigquery-public-data.covid19_open_data.covid19_open_data
            WHERE aggregation_level = 0''')

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,location_key,country_name,age_00_09_percentage,age_10_19_percentage,age_20_29_percentage,age_30_39_percentage,age_40_49_percentage,age_50_59_percentage,age_60_69_percentage,age_70_79_percentage,age_80_plus_percentage
0,AT,Austria,9.656177,9.733592,12.806184,13.581155,13.649923,15.605634,11.167300,8.804795,4.995239
1,BH,Bahrain,12.819533,10.687401,18.022630,29.838862,15.087186,8.241208,3.912357,1.047143,0.343680
2,BN,Brunei,15.167447,14.876921,15.931133,17.501937,15.544147,11.479532,6.517510,2.187285,0.794088
3,CA,Canada,10.521182,10.522239,13.503775,13.986114,12.841521,13.731152,12.486684,7.998154,4.409178
4,CL,Chile,13.816312,14.190728,17.045280,16.760006,14.671694,13.384947,10.195668,5.651109,3.059753
...,...,...,...,...,...,...,...,...,...,...,...
241,SA,Saudi Arabia,17.108746,13.960762,15.381112,20.050525,18.408725,9.239341,3.945327,1.418561,0.486901
242,SL,Sierra Leone,27.903838,23.407290,17.619614,12.749805,8.435443,5.254278,2.933753,1.364037,0.331942
243,VA,Vatican City,198.145859,145.735476,147.836836,152.781211,164.276885,166.625464,154.882571,146.971570,81.705810
244,VC,Saint Vincent and the Grenadines,14.373530,15.708401,15.775100,14.160816,13.054882,12.249092,8.270616,4.058695,2.348869


### 3.6.3 W jakich krajach uzupełniane są informacje o łóżkach szpitalnych, pielęgniarkach i lekarzach

In [14]:
query = ('''SELECT DISTINCT location_key, country_name
            FROM bigquery-public-data.covid19_open_data.covid19_open_data
            WHERE aggregation_level = 0 AND hospital_beds_per_1000 IS NOT NULL AND nurses_per_1000 IS NOT NULL AND physicians_per_1000 IS NOT NULL''')

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,location_key,country_name
0,MX,Mexico
1,DZ,Algeria
2,ID,Indonesia
3,CZ,Czech Republic
4,AF,Afghanistan
5,GR,Greece
6,AM,Armenia
7,FI,Finland
8,DK,Denmark
9,KR,South Korea


### 3.6.4 Jakie rodzaje szczepionek wyszczególniono w danych

In [15]:
li = [l for l in list(dfC.columns)
      if 'new_vaccine_doses_administered_' in l]
li = [l.replace('new_vaccine_doses_administered_', '') for l in li]

display(li)

['pfizer', 'moderna', 'janssen']

### 3.6.5 Które z państw uzupełniają wszystkie kluczowe informacje na temat zachorowań i śmierci z podziałem na płeć

In [16]:
query = ('''SELECT DISTINCT location_key, country_name
            FROM bigquery-public-data.covid19_open_data.covid19_open_data
            WHERE aggregation_level = 0 
            AND new_confirmed_male IS NOT NULL 
            AND new_confirmed_female IS NOT NULL 
            AND cumulative_confirmed_male IS NOT NULL 
            AND cumulative_confirmed_female IS NOT NULL 
            AND new_deceased_male IS NOT NULL 
            AND new_deceased_female IS NOT NULL 
            AND cumulative_deceased_male IS NOT NULL 
            AND cumulative_deceased_female IS NOT NULL''')

query_job = client.query(query)    
query_result = query_job.result()  
dfR = query_result.to_dataframe()

display(dfR)

,location_key,country_name
0,DE,Germany
1,PE,Peru
2,AR,Argentina
3,PH,Philippines
4,CO,Colombia
5,ES,Spain
6,BR,Brazil
7,IN,India
8,MX,Mexico
9,CZ,Czech Republic


# 4 Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.

## 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

- location_key
- country_code (iso_3)
- country_name
- population
- area_sq_km
- longitude
- latitude


In [17]:
query = ('''SELECT DISTINCT location_key, iso_3166_1_alpha_3, country_name, population, area_sq_km, longitude, latitude
            FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE aggregation_level = 0''')

query_job = client.query(query)    
query_result = query_job.result()  
country_df = query_result.to_dataframe()

country_df = country_df.rename(columns={'iso_3166_1_alpha_3': 'country_code'})

display(country_df)

,location_key,country_code,country_name,population,area_sq_km,longitude,latitude
0,AD,AND,Andorra,77265,470,1.555278,42.558333
1,AE,ARE,United Arab Emirates,9890400,83600,54.300000,24.400000
2,AN,None,Netherlands Antilles,197041,800,-66.000000,15.000000
3,BF,BFA,Burkina Faso,20903278,274220,-2.066667,12.266667
4,BH,BHR,Bahrain,1701583,778,50.551111,26.067500
...,...,...,...,...,...,...,...
241,TL,TLS,East Timor,1318442,14870,125.750000,-8.966667
242,TO,TON,Tonga,105697,750,-174.810278,-20.587778
243,XK,RKS,Kosovo,1883018,10909,20.833333,42.550000
244,MY,MYS,Malaysia,32655400,330345,102.314362,3.780511


## 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

- location_key
- date
- cumulative_confirmed
- new_confirmed
- cumulative_tested
- new_tested

In [18]:
query = ('''SELECT location_key, date, cumulative_confirmed, new_confirmed, cumulative_tested, new_tested
            FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE aggregation_level = 0 ORDER BY date''')

query_job = client.query(query)    
query_result = query_job.result()  
covid_df = query_result.to_dataframe()

display(covid_df)

,location_key,date,cumulative_confirmed,new_confirmed,cumulative_tested,new_tested
0,MO,2020-01-01,<NA>,<NA>,<NA>,None
1,MS,2020-01-01,0,0,<NA>,None
2,PW,2020-01-01,0,0,<NA>,None
3,TO,2020-01-01,<NA>,<NA>,<NA>,None
4,NR,2020-01-01,<NA>,<NA>,<NA>,None
...,...,...,...,...,...,...
243781,SR,2022-09-17,<NA>,<NA>,<NA>,None
243782,UG,2022-09-17,<NA>,<NA>,<NA>,None
243783,CC,2022-09-17,<NA>,<NA>,<NA>,None
243784,PR,2022-09-17,<NA>,<NA>,<NA>,None


## 4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

- location_key
- date
- cumulative_deceased
- new_deceased

In [19]:
query = ('''SELECT location_key, date, cumulative_deceased, new_deceased
            FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE aggregation_level = 0 ORDER BY date''')

query_job = client.query(query)    
query_result = query_job.result()  
death_df = query_result.to_dataframe()

display(death_df)

,location_key,date,cumulative_deceased,new_deceased
0,MO,2020-01-01,<NA>,<NA>
1,MS,2020-01-01,0,0
2,PW,2020-01-01,0,0
3,TO,2020-01-01,<NA>,<NA>
4,BA,2020-01-01,0,0
...,...,...,...,...
243781,QA,2022-09-17,<NA>,<NA>
243782,RO,2022-09-17,<NA>,<NA>
243783,AD,2022-09-17,<NA>,<NA>
243784,CX,2022-09-17,<NA>,<NA>


## 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

- location_key
- date
- cumulative_persons_fully_vaccinated
- new_persons_vaccinated

In [20]:
query = ('''SELECT location_key, date, cumulative_persons_vaccinated, new_persons_vaccinated
            FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE aggregation_level = 0 ORDER BY date''')

query_job = client.query(query)
query_result = query_job.result()  
vaccine_df = query_result.to_dataframe()

display(vaccine_df)

,location_key,date,cumulative_persons_vaccinated,new_persons_vaccinated
0,MO,2020-01-01,<NA>,<NA>
1,MS,2020-01-01,<NA>,<NA>
2,NR,2020-01-01,<NA>,<NA>
3,RS,2020-01-01,<NA>,<NA>
4,BB,2020-01-01,<NA>,<NA>
...,...,...,...,...
243781,IQ,2022-09-17,<NA>,<NA>
243782,KZ,2022-09-17,<NA>,<NA>
243783,BD,2022-09-17,<NA>,<NA>
243784,WS,2022-09-17,<NA>,<NA>


## 4.5. Chcemy zaobserwować trendy i zależności dotyczące warunków atmosferycznych w trakcie pandemii COVID-19.

- location_key
- date
- minimum_temperature_celsius
- maximum_temperature_celsius
- rainfall_mm
- relative_humidity

In [21]:
query = ('''SELECT location_key, date, minimum_temperature_celsius, maximum_temperature_celsius, rainfall_mm, relative_humidity
            FROM bigquery-public-data.covid19_open_data.covid19_open_data WHERE aggregation_level = 0 ORDER BY date''')

query_job = client.query(query)    
query_result = query_job.result()  
weather_df = query_result.to_dataframe()

all_nan_cols = weather_df.columns[weather_df.isna().all()].tolist()
weather_df.drop(columns=all_nan_cols, inplace=True)

display(weather_df)

,location_key,date,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,relative_humidity
0,PW,2020-01-01,23.888889,29.000000,4.572000,87.362202
1,TO,2020-01-01,23.444444,28.180556,6.032500,75.823676
2,BA,2020-01-01,-4.750000,3.694444,0.000000,70.656417
3,BN,2020-01-01,23.796296,30.916667,7.196667,81.600004
4,RE,2020-01-01,22.888889,30.055556,6.858000,87.767006
...,...,...,...,...,...,...
243781,IE,2022-09-17,NaN,NaN,NaN,NaN
243782,KR,2022-09-17,NaN,NaN,NaN,NaN
243783,SO,2022-09-17,NaN,NaN,NaN,NaN
243784,TL,2022-09-17,NaN,NaN,NaN,NaN


## Zapis do CSV

In [22]:
country_df.to_csv('country_df.csv', index=False)
covid_df.to_csv('covid_df.csv', index=False)
death_df.to_csv('death_df.csv', index=False)
vaccine_df.to_csv('vaccine_df.csv', index=False)
weather_df.to_csv('weather_df.csv', index=False)

# 5 Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [23]:
merged_df = pd.merge(country_df, covid_df, on='location_key', how='inner')

for df in [death_df, vaccine_df, weather_df]:
    merged_df = pd.merge(merged_df, df, on=['location_key', 'date'], how='inner')
display(merged_df)

,location_key,country_code,country_name,population,area_sq_km,longitude,latitude,date,cumulative_confirmed,new_confirmed,cumulative_tested,new_tested,cumulative_deceased,new_deceased,cumulative_persons_vaccinated,new_persons_vaccinated,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,relative_humidity
0,AD,AND,Andorra,77265,470,1.555278,42.558333,2020-01-01,0,0,<NA>,None,0,0,<NA>,<NA>,0.138889,8.208333,3.302000,72.773049
1,AD,AND,Andorra,77265,470,1.555278,42.558333,2020-01-02,0,0,<NA>,None,0,0,<NA>,<NA>,-0.722222,10.055556,6.688667,70.841314
2,AD,AND,Andorra,77265,470,1.555278,42.558333,2020-01-03,0,0,<NA>,None,0,0,<NA>,<NA>,0.597222,8.402778,5.016500,71.117248
3,AD,AND,Andorra,77265,470,1.555278,42.558333,2020-01-04,0,0,<NA>,None,0,0,<NA>,<NA>,1.125000,8.708333,3.810000,77.338640
4,AD,AND,Andorra,77265,470,1.555278,42.558333,2020-01-05,0,0,<NA>,None,0,0,<NA>,<NA>,-1.000000,11.361111,2.476500,60.762380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243781,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,2022-09-13,92357092,54413,<NA>,None,1004449,460,263103582,0,12.761111,33.616667,0.000000,49.999442
243782,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,2022-09-14,92440495,83403,<NA>,None,1005195,705,263103582,0,21.407407,27.049383,0.000000,51.394811
243783,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,2022-09-15,<NA>,<NA>,<NA>,None,<NA>,<NA>,263103582,0,NaN,NaN,NaN,NaN
243784,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,2022-09-16,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN


# Procentowy udział pustych wartości dla poszczególnych column

In [24]:
print((merged_df.isnull().sum()/243786)*100)

location_key                      0.000000
country_code                      0.406504
country_name                      0.000000
population                        0.406504
area_sq_km                        0.813008
longitude                         1.626016
latitude                          1.626016
date                              0.000000
cumulative_confirmed              6.565594
new_confirmed                     6.565184
cumulative_tested                66.916066
new_tested                       68.305399
cumulative_deceased               6.635738
new_deceased                      6.636148
cumulative_persons_vaccinated    78.694429
new_persons_vaccinated           77.549982
minimum_temperature_celsius       9.359028
maximum_temperature_celsius       9.362720
rainfall_mm                      10.440304
relative_humidity                10.101893
dtype: float64


Należy uzupełnić wartości w country_code

In [25]:
display(merged_df.query('country_code != country_code'))

# podany kraj nie posiada kodu w formacie iso_3

,location_key,country_code,country_name,population,area_sq_km,longitude,latitude,date,cumulative_confirmed,new_confirmed,cumulative_tested,new_tested,cumulative_deceased,new_deceased,cumulative_persons_vaccinated,new_persons_vaccinated,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,relative_humidity
1982,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2020-01-01,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
1983,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2020-01-02,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
1984,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2020-01-03,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
1985,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2020-01-04,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
1986,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2020-01-05,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2022-09-13,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
2969,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2022-09-14,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
2970,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2022-09-15,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN
2971,AN,None,Netherlands Antilles,197041,800,-66.0,15.0,2022-09-16,<NA>,<NA>,<NA>,None,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN


In [26]:
merged_df.to_csv('merged_df.csv', index=False)

# 6 Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach:

## 6.1 world_countries.csv

In [27]:
wcdf = pd.read_csv('world_countries.csv')
wcdf = wcdf.rename(columns={'CCA3': 'country_code'})
wcdf

,Rank,country_code,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56
3,213,ASM,American Samoa,Pago Pago,Oceania,44273,46189,51368,54849,58230,47818,32886,27075,199,222.4774,0.9831,0.00
4,203,AND,Andorra,Andorra la Vella,Europe,79824,77700,71746,71519,66097,53569,35611,19860,468,170.5641,1.0100,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,226,WLF,Wallis and Futuna,Mata-Utu,Oceania,11572,11655,12182,13142,14723,13454,11315,9377,142,81.4930,0.9953,0.00
230,172,ESH,Western Sahara,El Aaiún,Africa,575986,556048,491824,413296,270375,178529,116775,76371,266000,2.1654,1.0184,0.01
231,46,YEM,Yemen,Sanaa,Asia,33696614,32284046,28516545,24743946,18628700,13375121,9204938,6843607,527968,63.8232,1.0217,0.42
232,63,ZMB,Zambia,Lusaka,Africa,20017675,18927715,16248230,13792086,9891136,7686401,5720438,4281671,752612,26.5976,1.0280,0.25


In [28]:
# zostało zastosowane złącznie left, aby nie utracić danych z poprzednich kroków w krajach które nie pojawiają się w pliku world_countries.csv
merged_df_w_countries = pd.merge(merged_df, wcdf, on='country_code', how='left')
merged_df_w_countries = merged_df_w_countries.drop(columns=['population', 'area_sq_km'])
merged_df_w_countries

,location_key,country_code,country_name,longitude,latitude,date,cumulative_confirmed,new_confirmed,cumulative_tested,new_tested,...,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,AD,AND,Andorra,1.555278,42.558333,2020-01-01,0,0,<NA>,None,...,71746.0,71519.0,66097.0,53569.0,35611.0,19860.0,468.0,170.5641,1.0100,0.00
1,AD,AND,Andorra,1.555278,42.558333,2020-01-02,0,0,<NA>,None,...,71746.0,71519.0,66097.0,53569.0,35611.0,19860.0,468.0,170.5641,1.0100,0.00
2,AD,AND,Andorra,1.555278,42.558333,2020-01-03,0,0,<NA>,None,...,71746.0,71519.0,66097.0,53569.0,35611.0,19860.0,468.0,170.5641,1.0100,0.00
3,AD,AND,Andorra,1.555278,42.558333,2020-01-04,0,0,<NA>,None,...,71746.0,71519.0,66097.0,53569.0,35611.0,19860.0,468.0,170.5641,1.0100,0.00
4,AD,AND,Andorra,1.555278,42.558333,2020-01-05,0,0,<NA>,None,...,71746.0,71519.0,66097.0,53569.0,35611.0,19860.0,468.0,170.5641,1.0100,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243781,US,USA,United States of America,-98.579500,39.828175,2022-09-13,92357092,54413,<NA>,None,...,324607776.0,311182845.0,282398554.0,248083732.0,223140018.0,200328340.0,9372610.0,36.0935,1.0038,4.24
243782,US,USA,United States of America,-98.579500,39.828175,2022-09-14,92440495,83403,<NA>,None,...,324607776.0,311182845.0,282398554.0,248083732.0,223140018.0,200328340.0,9372610.0,36.0935,1.0038,4.24
243783,US,USA,United States of America,-98.579500,39.828175,2022-09-15,<NA>,<NA>,<NA>,None,...,324607776.0,311182845.0,282398554.0,248083732.0,223140018.0,200328340.0,9372610.0,36.0935,1.0038,4.24
243784,US,USA,United States of America,-98.579500,39.828175,2022-09-16,<NA>,<NA>,<NA>,None,...,324607776.0,311182845.0,282398554.0,248083732.0,223140018.0,200328340.0,9372610.0,36.0935,1.0038,4.24


## 6.2 GDP.csv

Ten plik zdecydowaliśmy się połączyć jedynie z dataframem'em zawierającym dane o krajach, ze względu na to, że przy łączeniu z dataframe'em zawierającym wszystkie informacje zaiwerał on aż 9 milionów wierszy przez to w jakis spsób wyglądają dane w tym pliku csv. W ten spsób dane są znacznie czytelniejsze 

In [29]:
gdpdf = pd.read_csv('GDP.csv')
gdpdf = gdpdf.rename(columns={'Country Code': 'country_code'})
gdpdf

,Country Name,country_code,Year,Value
0,Arab World,ARB,1968,2.576068e+10
1,Arab World,ARB,1969,2.843420e+10
2,Arab World,ARB,1970,3.138550e+10
3,Arab World,ARB,1971,3.642691e+10
4,Arab World,ARB,1972,4.331606e+10
...,...,...,...,...
11502,Zimbabwe,ZWE,2012,1.424249e+10
11503,Zimbabwe,ZWE,2013,1.545177e+10
11504,Zimbabwe,ZWE,2014,1.589105e+10
11505,Zimbabwe,ZWE,2015,1.630467e+10


In [30]:
countries_gdp = pd.merge(country_df, gdpdf, on='country_code', how='inner')
countries_gdp

,location_key,country_code,country_name,population,area_sq_km,longitude,latitude,Country Name,Year,Value
0,AD,AND,Andorra,77265,470,1.555278,42.558333,Andorra,1970,7.861921e+07
1,AD,AND,Andorra,77265,470,1.555278,42.558333,Andorra,1971,8.940982e+07
2,AD,AND,Andorra,77265,470,1.555278,42.558333,Andorra,1972,1.134082e+08
3,AD,AND,Andorra,77265,470,1.555278,42.558333,Andorra,1973,1.508201e+08
4,AD,AND,Andorra,77265,470,1.555278,42.558333,Andorra,1974,1.865587e+08
...,...,...,...,...,...,...,...,...,...,...
9169,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,United States,2012,1.615526e+13
9170,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,United States,2013,1.669152e+13
9171,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,United States,2014,1.739310e+13
9172,US,USA,United States of America,331002647,9831510,-98.579500,39.828175,United States,2015,1.812071e+13


In [31]:
merged_df_w_countries.to_csv('merged_with_world_countries.csv', index=False)
countries_gdp.to_csv('gdp_country.csv', index=False)

# 7 Więcej danych

## 7.1 Zbiór 1 - % ludzi bezrobotnych na przestrzeni lat, te dane łączymy z informacjami o GDP

In [32]:
unratedf = pd.read_csv('unemployment-rate.csv')
unratedf = unratedf.rename(columns={'Entity': 'country_name', 'Code': 'country_code'})
display(unratedf)

,country_name,country_code,Year,"Unemployment, total (% of total labor force) (modeled ILO estimate)"
0,Afghanistan,AFG,1991,10.649
1,Afghanistan,AFG,1992,10.821
2,Afghanistan,AFG,1993,10.723
3,Afghanistan,AFG,1994,10.726
4,Afghanistan,AFG,1995,11.179
...,...,...,...,...
6226,Zimbabwe,ZWE,2017,4.785
6227,Zimbabwe,ZWE,2018,4.796
6228,Zimbabwe,ZWE,2019,4.833
6229,Zimbabwe,ZWE,2020,5.351


In [33]:
gdp_unemployment = pd.merge(countries_gdp, unratedf, on=['country_code', 'country_name', 'Year'], how='inner')
gdp_unemployment = gdp_unemployment.rename(columns={'Value': 'GDP'})
display(gdp_unemployment)

,location_key,country_code,country_name,population,area_sq_km,longitude,latitude,Country Name,Year,GDP,"Unemployment, total (% of total labor force) (modeled ILO estimate)"
0,AE,ARE,United Arab Emirates,9890400,83600,54.300000,24.400000,United Arab Emirates,1991,5.155217e+10,1.866
1,AE,ARE,United Arab Emirates,9890400,83600,54.300000,24.400000,United Arab Emirates,1992,5.423917e+10,1.842
2,AE,ARE,United Arab Emirates,9890400,83600,54.300000,24.400000,United Arab Emirates,1993,5.562517e+10,1.849
3,AE,ARE,United Arab Emirates,9890400,83600,54.300000,24.400000,United Arab Emirates,1994,5.930509e+10,1.806
4,AE,ARE,United Arab Emirates,9890400,83600,54.300000,24.400000,United Arab Emirates,1995,6.574367e+10,1.800
...,...,...,...,...,...,...,...,...,...,...,...
4364,MY,MYS,Malaysia,32655400,330345,102.314362,3.780511,Malaysia,2012,3.144431e+11,3.100
4365,MY,MYS,Malaysia,32655400,330345,102.314362,3.780511,Malaysia,2013,3.232772e+11,3.160
4366,MY,MYS,Malaysia,32655400,330345,102.314362,3.780511,Malaysia,2014,3.380620e+11,2.880
4367,MY,MYS,Malaysia,32655400,330345,102.314362,3.780511,Malaysia,2015,2.964340e+11,3.100


## 7.2 Zbiór 2 - ceny złota

In [34]:
golddf = pd.read_csv('Gold.csv')
golddf = golddf.add_suffix('_Gold')
golddf = golddf.rename(columns={'Date_Gold': 'date'})
def modify(x):
    month = x.split('/')[0]
    if len(month) == 1:
        month = '0' + month
    day = x.split('/')[1]
    if len(day) == 1:
        day = '0' + day
    year = x.split('/')[2]
    return str(year + '-' + month + '-' + day)
golddf['date'] = golddf['date'].apply(lambda x: modify(x))
golddf['date'] = golddf['date'].astype(str)
display(golddf)

,date,Open_Gold,High_Gold,Low_Gold,Close_Gold,Adj Close_Gold,Volume_Gold
0,2023-03-29,1966.1,1966.1,1966.1,1966.1,1966.1,21.0
1,2023-03-28,1972.4,1972.4,1972.4,1972.4,1972.4,21.0
2,2023-03-27,1957.2,1957.2,1952.4,1952.4,1952.4,877.0
3,2023-03-24,1991.7,1995.4,1982.1,1982.1,1982.1,19.0
4,2023-03-23,1990.5,1994.6,1990.5,1993.8,1993.8,33.0
...,...,...,...,...,...,...,...
1227,2018-04-06,1326.5,1333.0,1319.0,1331.9,1331.9,348.0
1228,2018-04-05,1333.5,1333.7,1322.6,1324.3,1324.3,258.0
1229,2018-04-04,1331.8,1347.3,1331.6,1335.8,1335.8,718.0
1230,2018-04-03,1340.4,1341.5,1328.5,1332.8,1332.8,702.0


In [35]:
merged_df_w_countries['date'] = merged_df_w_countries['date'].astype(str)
merged_gold = pd.merge(merged_df_w_countries, golddf, on='date', how='left')
merged_gold

,location_key,country_code,country_name,longitude,latitude,date,cumulative_confirmed,new_confirmed,cumulative_tested,new_tested,...,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Open_Gold,High_Gold,Low_Gold,Close_Gold,Adj Close_Gold,Volume_Gold
0,AD,AND,Andorra,1.555278,42.558333,2020-01-01,0,0,<NA>,None,...,468.0,170.5641,1.0100,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,AND,Andorra,1.555278,42.558333,2020-01-02,0,0,<NA>,None,...,468.0,170.5641,1.0100,0.00,1518.1,1528.7,1518.0,1524.5,1524.5,214.0
2,AD,AND,Andorra,1.555278,42.558333,2020-01-03,0,0,<NA>,None,...,468.0,170.5641,1.0100,0.00,1530.1,1552.7,1530.1,1549.2,1549.2,107.0
3,AD,AND,Andorra,1.555278,42.558333,2020-01-04,0,0,<NA>,None,...,468.0,170.5641,1.0100,0.00,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,AND,Andorra,1.555278,42.558333,2020-01-05,0,0,<NA>,None,...,468.0,170.5641,1.0100,0.00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243781,US,USA,United States of America,-98.579500,39.828175,2022-09-13,92357092,54413,<NA>,None,...,9372610.0,36.0935,1.0038,4.24,1720.5,1720.5,1697.3,1705.0,1705.0,432.0
243782,US,USA,United States of America,-98.579500,39.828175,2022-09-14,92440495,83403,<NA>,None,...,9372610.0,36.0935,1.0038,4.24,1698.7,1701.8,1696.5,1696.5,1696.5,532.0
243783,US,USA,United States of America,-98.579500,39.828175,2022-09-15,<NA>,<NA>,<NA>,None,...,9372610.0,36.0935,1.0038,4.24,1685.5,1686.0,1662.3,1665.4,1665.4,1125.0
243784,US,USA,United States of America,-98.579500,39.828175,2022-09-16,<NA>,<NA>,<NA>,None,...,9372610.0,36.0935,1.0038,4.24,1660.4,1674.9,1651.7,1671.7,1671.7,464.0


## 7.3 Zbiór 3 - Akcje Microsoftu

In [36]:
msdf = pd.read_csv('microsoft_stocks.csv')
msdf = msdf.add_suffix('_MS')
msdf = msdf.rename(columns={'Date_MS': 'date'})
msdf['date'] = msdf['date'].astype(str)
display(msdf)

,date,High_MS,Low_MS,Open_MS,Close_MS,Volume_MS,Adj Close_MS
0,2015-01-12,47.540001,46.360001,47.419998,46.599998,23651900.0,40.786396
1,2015-01-13,47.910000,46.060001,46.970001,46.360001,35270600.0,40.576340
2,2015-01-14,46.240002,45.619999,45.959999,45.959999,29719600.0,40.226257
3,2015-01-15,46.380001,45.410000,46.220001,45.480000,32750800.0,39.806129
4,2015-01-16,46.279999,45.169998,45.310001,46.240002,35695300.0,40.471313
...,...,...,...,...,...,...,...
1920,2022-08-26,280.339996,267.980011,279.079987,268.089996,27549300.0,268.089996
1921,2022-08-29,267.399994,263.850006,265.850006,265.230011,20338500.0,265.230011
1922,2022-08-30,267.049988,260.660004,266.670013,262.970001,22767100.0,262.970001
1923,2022-08-31,267.109985,261.329987,265.390015,261.470001,24791800.0,261.470001


In [37]:
merged_ms = pd.merge(merged_gold, msdf, on='date', how='left')
merged_ms

,location_key,country_code,country_name,longitude,latitude,date,cumulative_confirmed,new_confirmed,cumulative_tested,new_tested,...,Low_Gold,Close_Gold,Adj Close_Gold,Volume_Gold,High_MS,Low_MS,Open_MS,Close_MS,Volume_MS,Adj Close_MS
0,AD,AND,Andorra,1.555278,42.558333,2020-01-01,0,0,<NA>,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,AND,Andorra,1.555278,42.558333,2020-01-02,0,0,<NA>,None,...,1518.0,1524.5,1524.5,214.0,160.729996,158.330002,158.779999,160.619995,22622100.0,156.591980
2,AD,AND,Andorra,1.555278,42.558333,2020-01-03,0,0,<NA>,None,...,1530.1,1549.2,1549.2,107.0,159.949997,158.059998,158.320007,158.619995,21116200.0,154.642151
3,AD,AND,Andorra,1.555278,42.558333,2020-01-04,0,0,<NA>,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,AND,Andorra,1.555278,42.558333,2020-01-05,0,0,<NA>,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243781,US,USA,United States of America,-98.579500,39.828175,2022-09-13,92357092,54413,<NA>,None,...,1697.3,1705.0,1705.0,432.0,NaN,NaN,NaN,NaN,NaN,NaN
243782,US,USA,United States of America,-98.579500,39.828175,2022-09-14,92440495,83403,<NA>,None,...,1696.5,1696.5,1696.5,532.0,NaN,NaN,NaN,NaN,NaN,NaN
243783,US,USA,United States of America,-98.579500,39.828175,2022-09-15,<NA>,<NA>,<NA>,None,...,1662.3,1665.4,1665.4,1125.0,NaN,NaN,NaN,NaN,NaN,NaN
243784,US,USA,United States of America,-98.579500,39.828175,2022-09-16,<NA>,<NA>,<NA>,None,...,1651.7,1671.7,1671.7,464.0,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
gdp_unemployment.to_csv('gdp_unemployment.csv', index=False)
merged_ms.to_csv('covid_ms_gold.csv', index=False)